<a href="https://colab.research.google.com/github/prathameshks/FileChat-Using-RAG/blob/main/fileChat-with-RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests

Downloading The Document (PDF File)

In [ ]:
# get document name
pdf_name = "human-nutrition-text.pdf"

if not os.path.exists(pdf_name):
    print(f"Downloading {pdf_name}")

    # url of document to download

    pdf_url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    resp = requests.get(pdf_url)

    if resp.status_code == 200:
        with open(pdf_name, "wb") as f:
            f.write(resp.content)
        print(f"Downloaded {pdf_name} and Saved")
    else:
        print(f"Failed to download {pdf_name}")
else:
    print(f"{pdf_name} already exists")


Downloaded human-nutrition-text.pdf and Saved


Open the PDF

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 34.6 MB/s eta 0:00:00


In [ ]:
!pip install spacy

In [ ]:
import fitz  # PyMuPDF !pip install pymupdf
from tqdm.auto import tqdm

def text_formatter(text:str)->str:
    """perform basic text preprocessing. """

    cleaned_text = text.replace("\n", " ").strip()

    return cleaned_text


def open_and_read_pdf(pdf_name:str)->list[dict]:
    doc = fitz.open(pdf_name)
    pages_and_textx = []

    for page_no,page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_textx.append({
            "page_number" : page_no+1,
            "page_char_count" : len(text),
            "page_word_count" : len(text.split()),
            "page_sentence_count_raw" : len(text.split(". ")),
            "page_token_count" : len(text)/4,
            "text" : text
            })
    return pages_and_textx

pages_and_texts = open_and_read_pdf(pdf_name)

In [ ]:
import random

random.sample(pages_and_texts,k=3)

[{'page_number': 792,
  'page_char_count': 839,
  'page_word_count': 138,
  'page_sentence_count_raw': 14,
  'page_token_count': 209.75,
  'text': 'Meal  Calories Total Meal/Snack  Calories  Breakfast  1 scrambled egg  92  \xa0\xa0\xa0with sliced mushrooms and spinach  7  ½ whole-wheat muffin  67  1 tsp. margarine-like spread  15  1 orange  65  8 oz. low-sodium tomato juice  53  299  Snack  6 oz. fat-free flavored yogurt  100  \xa0\xa0\xa0with ½ c. raspberries  32  132  Lunch  1 sandwich on pumpernickel bread  160  \xa0\xa0\xa0with smoked turkey deli meat,  30  \xa0\xa0\xa04 slices tomato  14  \xa0\xa0\xa02 lettuce leaves  3  \xa0\xa0\xa01 tsp. mustard  3  1 oz. baked potato chips  110  ½ c. blueberries, with 1 tsp. sugar  57  8 oz. fat-free milk  90  467  Snack  1 banana  105  7 reduced-fat high-fiber crackers  120  225  Dinner  1 c. Greek salad (tomatoes, cucumbers,  feta)  150  \xa0\xa0\xa0with 5 Greek olives,  45  \xa0\xa0\xa0with 1.5 tsp. olive oil  60  3 oz. grilled chicken breas

In [ ]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,29,4,1,7.25,Human Nutrition: 2020 Edition
1,2,0,0,1,0.00,
2,3,320,42,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,4,212,30,1,53.00,Human Nutrition: 2020 Edition by University of...
4,5,797,116,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [ ]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,604.50000,1148.004139,171.966060,9.972682,287.001035
std,348.86387,560.382275,86.491465,6.187226,140.095569
min,1.00000,0.000000,0.000000,1.000000,0.000000
25%,302.75000,762.000000,109.000000,4.000000,190.500000
50%,604.50000,1231.500000,183.000000,10.000000,307.875000
75%,906.25000,1603.500000,239.000000,14.000000,400.875000
max,1208.00000,2308.000000,393.000000,32.000000,577.000000


Why we need token Count?

1. Embeding Model don't deal with infinite tokens.
2. LLMs don't deal with lnfinite tokens.

# Splitting Pages into sentences

Two days to do this
1. split by `". "` OR `".\n"`
2. Use NLP like spaCy or NLTK

In [ ]:
from spacy.lang.en import English

nlp = English()

# add a sentencilizer pipeline
nlp.add_pipe("sentencizer")

# create a document example
doc = nlp("THis is sentence1. I like cats")

list(doc.sents)

[THis is sentence1., I like cats]

In [ ]:
pages_and_texts[600]

{'page_number': 601,
 'page_char_count': 863,
 'page_word_count': 117,
 'page_sentence_count_raw': 8,
 'page_token_count': 215.75,
 'text': 'Image by  Allison  Calabrese /  CC BY 4.0  Korsakoff syndrome can cause similar symptoms as beriberi such  as confusion, loss of coordination, vision changes, hallucinations,  and may progress to coma and death. This condition is specific  to alcoholics as diets high in alcohol can cause thiamin deficiency.  Other individuals at risk include individuals who also consume diets  typically low in micronutrients such as those with eating disorders,  elderly, and individuals who have gone through gastric bypass  surgery.5  Figure 9.10 The Role of Thiamin  Figure 9.11 Beriberi, Thiamin Deficiency  5.\xa0Fact Sheets for Health Professionals: Thiamin. National  Institute of Health, Office of Dietary Supplements.  \xa0https://ods.od.nih.gov/factsheets/Thiamin- HealthProfessional/. Updated Feburary 11, 2016.  Accessed October 22, 2017.  Water-Soluble Vitami

In [ ]:
from tqdm import tqdm

for page in tqdm(pages_and_texts):
    page["sentences"] = [str(sent) for sent in nlp(page['text']).sents]

    page["page_sentence_count"] = len(page["sentences"])

100%|██████████| 1208/1208 [00:02<00:00, 474.02it/s]


In [ ]:
pages_and_texts[103]

{'page_number': 104,
 'page_char_count': 1693,
 'page_word_count': 264,
 'page_sentence_count_raw': 13,
 'page_token_count': 423.25,
 'text': 'Basic Biology, Anatomy, and  Physiology  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  The Basic Structural and Functional Unit of Life:  The Cell  What distinguishes a living\xa0organism from an inanimate object? A  living organism conducts self-sustaining biological processes. A cell  is the smallest and most basic form of life.  The cell theory incorporates three principles:  Cells are the most basic building units of life.\xa0All living things  are composed of cells. New cells are made from preexisting cells,  which divide in two. Who you are has been determined because  of two cells that came together inside your mother’s womb. The  two cells containing all of your genetic information (DNA) united to  begin making new life. Cells divided and differentiated into other  cells with specif

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,604.50000,1148.004139,171.966060,9.972682,287.001035,10.319536
std,348.86387,560.382275,86.491465,6.187226,140.095569,6.300843
min,1.00000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,302.75000,762.000000,109.000000,4.000000,190.500000,5.000000
50%,604.50000,1231.500000,183.000000,10.000000,307.875000,10.000000
75%,906.25000,1603.500000,239.000000,14.000000,400.875000,15.000000
max,1208.00000,2308.000000,393.000000,32.000000,577.000000,28.000000


## Chunking the sentences together

here we are getting in 10 sentences at a time.

Framework like langchain can do this task easily.

## Need of Chunking
1. makes easy to filter
2. context of embedding model
2. context of llm

In [ ]:
# define number of sentences chunk
sentences_chunk_size = 10

# actual function to split list of sentences to list of list of size <= 10 of sentences

def split_sentence_chunk(sentence_list:list[str],size_chunk:int = sentences_chunk_size) -> list[list[str]]:
    return [sentence_list[i:i + size_chunk] for i in range(0,len(sentence_list),size_chunk)]

test  = list(range(1,26))
print(test)
split_sentence_chunk(test)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 [21, 22, 23, 24, 25]]

In [ ]:
# make chunks of original data
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_sentence_chunk(item["sentences"],sentences_chunk_size)
    item["chunk_count"] = len(item["sentence_chunks"])

100%|██████████| 1208/1208 [00:00<00:00, 227309.07it/s]


In [ ]:
random.sample(pages_and_texts,k=1)

[{'page_number': 307,
  'page_char_count': 1796,
  'page_word_count': 264,
  'page_sentence_count_raw': 17,
  'page_token_count': 449.0,
  'text': 'a diet high in fructose could potentially stimulate fat deposition and  weight gain.  In human studies, excessive fructose intake has sometimes been  associated with weight gain, but results are inconsistent. Moderate  fructose intake is not associated with weight gain at all. Moreover,  other studies show that some fructose in the diet actually improves  glucose metabolism especially in people with Type 2 diabetes.5  In fact, people with diabetes were once advised to use fructose  as an alternative sweetener to table sugar. Overall, there is no good  evidence that moderate fructose consumption contributes to  weight gain and chronic disease. At this time conclusive evidence  is not available on whether fructose is any worse than any other  added sugar in increasing the risk for obesity, Type 2 diabetes, and  cardiovascular disease.  Do Low

In [ ]:
df=pd.DataFrame(pages_and_texts)
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count,chunk_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,604.50000,1148.004139,171.966060,9.972682,287.001035,10.319536,1.525662
std,348.86387,560.382275,86.491465,6.187226,140.095569,6.300843,0.644397
min,1.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,302.75000,762.000000,109.000000,4.000000,190.500000,5.000000,1.000000
50%,604.50000,1231.500000,183.000000,10.000000,307.875000,10.000000,1.000000
75%,906.25000,1603.500000,239.000000,14.000000,400.875000,15.000000,2.000000
max,1208.00000,2308.000000,393.000000,32.000000,577.000000,28.000000,3.000000


## Splitting chunks to separate items
so that we can add meta data to each chunk ans can embed them together.


In [ ]:
pages_and_chunks = []

for item in tqdm(pages_and_texts):
    for chunk in item["sentence_chunks"]:
        chunk_dict = {}

        chunk_dict["page_number"] = item["page_number"] # original page number
        chunk_dict["text"] =  " ".join(chunk).replace("  "," ").strip() # original text
        chunk_dict["sentence_chunk"] = chunk
        chunk_dict["chunk_char_count"]  =  len(chunk_dict["text"])
        chunk_dict["chunk_word_count"]  =  len([word for word in chunk_dict["text"].split() if word])
        chunk_dict["chunk_token_count"] = chunk_dict["chunk_char_count"] / 4

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

100%|██████████| 1208/1208 [00:00<00:00, 31558.51it/s]


1843

In [ ]:
random.sample(pages_and_chunks,k=1)

[{'page_number': 210,
  'text': 'Image by Allison Calabrese / CC BY 4.0 The Kidneys Detect Blood Volume The kidneys are two bean-shaped organs, each about the size of a fist and located on either side of the spine just below the rib cage. The kidneys filter about 190 liters of blood and produce (on average) 1.5 liters of urine per day. Urine is mostly water, but it also contains electrolytes and waste products, such as urea. The amount of water filtered from the blood and excreted as urine is dependent on the amount of water in, and the electrolyte composition in the blood. Kidneys have protein sensors that detect blood volume from the pressure, or stretch, in the blood vessels of the kidneys. When blood volume is low, kidney cells detect decreased pressure and secrete the enzyme, renin. Renin travels in the blood and cleaves another protein into the active hormone, angiotensin. Angiotensin targets three different organs (the adrenal glands, the hypothalamus, and 168 | Regulation of Wa

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,625.38,735.11,112.85,183.78
std,347.79,447.64,71.28,111.91
min,1.00,12.00,3.00,3.00
25%,322.50,315.00,45.00,78.75
50%,628.00,747.00,115.00,186.75
75%,932.00,1119.00,173.00,279.75
max,1208.00,1832.00,298.00,458.00


# remove chunks with less than 30 token

In [ ]:
min_chunk_token_limit = 30

pages_and_chunks_filtered = df[df['chunk_token_count'] > min_chunk_token_limit].to_dict(orient="records")

In [ ]:
random.sample(pages_and_chunks_filtered,k=1)

[{'page_number': 119,
  'text': 'From the Small Intestine to the Large Intestine The process of digestion is fairly efficient. Any food that is still incompletely broken down (usually less than ten percent of food consumed) and the food’s indigestible fiber content move from the small intestine to the large intestine (colon) through a connecting valve. A main task of the large intestine is to absorb much of the remaining water. Remember, water is present not only in solid foods and beverages, but also the stomach releases a few hundred milliliters of gastric juice, and the pancreas adds approximately 500 milliliters during the digestion of the meal. For the body to conserve water, it is important that excessive water is not lost in fecal matter. In the large intestine, no further chemical or mechanical breakdown of food takes place unless it is accomplished by the bacteria that inhabit this portion of the intestinal tract. The number of bacteria residing in the large intestine is estim

## Creating Embeddings
using sentence transformers all-mpnet-base-v2 model on each chunk and save embeddings

In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.3 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import  SentenceTransformer
import torch

model = SentenceTransformer("all-mpnet-base-v2",
                            device="cuda" if torch.cuda.is_available() else "cpu")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
sample_sentences = ["This is learing RAG",
                    "Why sky is blue.",
                    "to be or not to be?"]
sample_embeddings = model.encode(sample_sentences)

for sent,emb in zip(sample_sentences,sample_embeddings):
    print(f"Sentence: {sent} \n Embedding: {emb}\n")


Sentence: This is learing RAG 
 Embedding: [ 1.35656400e-02 -4.05066721e-02  1.36057632e-02  3.92129757e-02
  3.16675603e-02 -1.27551369e-02 -2.19453145e-02  2.08696760e-02
 -2.45429892e-02 -1.30132651e-02  9.16951243e-03 -2.29534004e-02
  3.19749080e-02 -8.51907209e-02  4.52552848e-02 -7.22494675e-03
  4.48949113e-02  1.78468216e-03 -3.15136323e-03 -3.51242051e-02
  1.63251292e-02  1.83048453e-02  6.22125389e-03  1.84313599e-02
 -1.61896516e-02 -1.07237492e-02  1.45885972e-02  1.40310097e-02
 -1.98288634e-02  2.00343058e-02  1.25578074e-02 -1.09815923e-02
 -2.65760371e-03  9.31028277e-03  1.89347054e-06  3.10882600e-03
 -8.45359080e-03 -1.99653059e-02 -4.94666770e-02  5.83847389e-02
  5.50031513e-02  9.32039469e-02 -5.20484783e-02  1.68342739e-02
 -2.48698890e-03 -4.85586002e-02  1.12593984e-02  2.39718091e-02
  2.45057940e-02  5.45430183e-02  1.80712212e-02 -9.34032910e-03
  1.38299204e-02 -2.92354785e-02  1.50079474e-01  1.19404891e-03
 -5.37780672e-02  1.22232720e-01  8.64282437e-0

In [ ]:
%%time

sample_embeddings.shape

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


(3, 768)

In [ ]:
text_chunks = [item["text"] for item in pages_and_chunks_filtered]

text_chunks[419]

'often. • Calm your “sweet tooth” by eating fruits, such as berries or an apple. • Replace sugary soft drinks with seltzer water, tea, or a small amount of 100 percent fruit juice added to water or soda water. The Food Industry: Functional Attributes of Carbohydrates and the Use of Sugar Substitutes In the food industry, both fast-releasing and slow-releasing carbohydrates are utilized to give foods a wide spectrum of functional attributes, including increased sweetness, viscosity, bulk, coating ability, solubility, consistency, texture, body, and browning capacity. The differences in chemical structure between the different carbohydrates confer their varied functional uses in foods. Starches, gums, and pectins are used as thickening agents in making jam, cakes, cookies, noodles, canned products, imitation cheeses, and a variety of other foods. Molecular gastronomists use slow- releasing carbohydrates, such as alginate, to give shape and texture to their fascinating food creations. Add

In [ ]:

%%time
# train on CPU
# model.to("cpu")

# train on GPU of intel
model.to("cuda")

# train on GPU of amd
# model.to("rocm") #not working

for item in tqdm(pages_and_chunks_filtered):
    item["embedding"] = model.encode(item["text"])

# nearly 1 item/s

100%|██████████| 1680/1680 [00:31<00:00, 53.76it/s]

CPU times: user 30.4 s, sys: 217 ms, total: 30.6 s
Wall time: 31.3 s


In [ ]:
%%time

#embed all text in batches

text_chunk_embeddings = model.encode(text_chunks,
                                     batch_size=32,
                                     convert_to_tensor=True)



CPU times: user 23.7 s, sys: 86 ms, total: 23.8 s
Wall time: 23.9 s


In [ ]:
random.sample(pages_and_chunks_filtered,k=2)

[{'page_number': 623,
  'text': 'Age Group RDA Males and Females mcg DFE/day UL Infants (0–6 months) 65* Not possible to determine Infants (7–12 months) 80* Not possible to determine Children (1–3 years) 150 300 Children (4–8 years) 200 400 Children (9–13 years) 300 600 Adolescents (14–18 years) 400 800 Adults (> 19 years) 400 1000 *denotes Adequate Intake Dietary Supplement Fact Sheet: Folate. National Institute of Health, Office of Dietary Supplements. https://ods.od.nih.gov/factsheets/ Folate-HealthProfessional/. Updated April 20, 2016. Accessed October 22, 2017. Dietary Sources Folate is found naturally in a wide variety of food especially in dark leafy vegetables, fruits, and animal products. The U.S. Food and Drug Administration (FDA) began requiring manufacturers to fortify enriched breads, cereals, flours, and cornmeal to increase the consumption of folate in the American diet. For the folate content of various foods, see Table 9.26 “Folate Content of Various Foods”. Table 9.26

Save to file

In [ ]:
# 2.15 saving to file
text_chunks_with_embedding = pd.DataFrame(pages_and_chunks_filtered)

embedding_save_path = "text_chunks_with_embedding.csv"

text_chunks_with_embedding.to_csv(embedding_save_path,index=False)


# EMBEDDINGS SAVED CREATE READ MODE

In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.0 MB/s eta 0:00:00


In [2]:
# imports libraries so no need to run above cell
import os
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import requests
import torch

embedding_save_path = "text_chunks_with_embedding.csv"


In [ ]:
# if colab then install torch_xla
if 'COLAB_GPU' in os.environ:
    !pip install torch_xla

In [3]:
import torch
import platform
import os

def get_device():
    """make a device variable to automatically detect if GPU(cuda and ryzen both), Colab CPU,GPU,TPU else CPU at end"""
    # Check for CUDA GPU
    if torch.cuda.is_available():
        return torch.device("cuda")

    # Check for AMD GPU (ROCm)
    if hasattr(torch, 'has_rocm') and torch.has_rocm:
        return torch.device("rocm")

    # Check for Google Colab
    if 'COLAB_GPU' in os.environ:
        if os.environ['COLAB_GPU'] == '1':
            return torch.device("cuda")
        elif 'COLAB_TPU_ADDR' in os.environ:
            import torch_xla.core.xla_model as xm
            return xm.xla_device()

    # If none of the above, use CPU
    return torch.device("cpu")

# Get the device
device = get_device()
print(f"Using device: {device}")

# Optional: Print more details about the device
if device.type == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
elif device.type == "rocm":
    print("AMD GPU detected")
elif device.type == "xla":
    print("TPU detected")
else:
    print(f"CPU: {platform.processor()}")

Using device: cuda
GPU: Tesla T4


In [4]:
# read saved file using path
embedding_file_url = "https://github.com/prathameshks/FileChat-Using-RAG/raw/refs/heads/main/text_chunks_with_embedding.csv"

if(not os.path.exists(embedding_save_path)):
    # download from url
    print(f"Downloading {embedding_save_path}")
    embedding_file = requests.get(embedding_file_url)

    with open(embedding_save_path, "wb") as f:
        f.write(embedding_file.content)

    print(f"Downloaded {embedding_save_path} and Saved")
else:
    print("Found Embedding file.")

text_chunks_with_embedding = pd.read_csv(embedding_save_path)

# convert embedding column to np array from string
text_chunks_with_embedding['embedding'] = text_chunks_with_embedding['embedding'].apply(lambda x: np.fromstring(x.strip('[]'),sep=" "))

pages_and_chunks_filtered = text_chunks_with_embedding.to_dict(orient="records")


Downloaded text_chunks_with_embedding.csv and Saved


In [5]:
# convert embedding column to tensor from object
embeddings = torch.tensor(np.array(text_chunks_with_embedding["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([1680, 768])

In [6]:
text_chunks_with_embedding.dtypes

,0
page_number,int64
text,object
sentence_chunk,object
chunk_char_count,int64
chunk_word_count,int64
chunk_token_count,float64
embedding,object


In [7]:
random.sample(pages_and_chunks_filtered,k=1)

[{'page_number': 313,
  'text': 'Carbohydrates and Personal Diet Choices UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM In this chapter, you learned what carbohydrates are, the different types of carbohydrates in your diet, and that excess consumption of some types of carbohydrates cause disease while others decrease disease risk. Now that we know the benefits of eating the right carbohydrate, we will examine exactly how much should be eaten to promote health and prevent disease. How Many Carbohydrates Does a Person Need? The Food and Nutrition Board of IOM has set the Recommended Dietary Allowance (RDA) of carbohydrates for children and adults at 130 grams per day. This is the average minimum amount the brain requires to function properly. The Acceptable Macronutrient Distribution Range (AMDR) for carbohydrates is between 45 and 65 percent of your total caloric daily intake. This means that on a 2,000 kilocalorie diet, a person shou

In [8]:
from sentence_transformers import SentenceTransformer,util


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# create a query and its embedding
query = "Diseases Involving Proteins"
print(f"Query: {query}")
query_embedding = embedding_model.encode(query, convert_to_tensor=True)


Query: Diseases Involving Proteins


In [11]:
%time

# Find the closest 5 sentences of the corpus for query sentence based on cosine similarity
dot_scores = util.dot_score(a=query_embedding,b=embeddings)[0]


# showing top 5 results
top_5_dot_products = torch.topk(dot_scores,k=5)
top_5_dot_products

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 16.9 µs


torch.return_types.topk(
values=tensor([0.7183, 0.6313, 0.6193, 0.6077, 0.5601], device='cuda:0'),
indices=tensor([586, 587, 589, 599, 590], device='cuda:0'))

In [12]:
import textwrap

def print_wrap(text,wrap_width=80):
    print(textwrap.fill(text,wrap_width))

In [13]:
# show the data from data frame for top results

print(f"Query: {query} \n Top 5 Results:")

for score,index in zip(top_5_dot_products.values,top_5_dot_products.indices):
    print(f"Score: {score}")
    print("Text:")
    print_wrap(pages_and_chunks_filtered[index]["text"])

    print(f"\nPage No. {pages_and_chunks_filtered[index]['page_number']}")
    print("\n")

Query: Diseases Involving Proteins 
 Top 5 Results:
Score: 0.7182703614234924
Text:
Diseases Involving Proteins UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND
HUMAN NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM As you may recall,
moderation refers to having the proper amount of a nutrient—having neither too
little nor too much. A healthy diet incorporates all nutrients in moderation.
Low protein intake has several health consequences, and a severe lack of protein
in the diet eventually causes death. Although severe protein deficiency is a
rare occurrence in children and adults in the United States, it is estimated
that more than half of the elderly in nursing homes are protein-deficient. The
Acceptable Macronutrient Distribution Range (AMDR) for protein for adults is
between 10 and 35 percent of kilocalories, which is a fairly wide range. The
percent of protein in the diet that is associated with malnutrition and its
health consequences is less than 10 percent, but this is often accom

Cosine and Dot Similarity

In [14]:
def dot_product(vector1,vector2):
    return torch.dot(vector1,vector2)

def cosine_similarity(vector1,vector2):
    dot_p = torch.dot(vector1,vector2)

    # euclidian magnitude
    # mag_1 = torch.norm(vector1)
    mag_1 = torch.sqrt(torch.sum(vector1**2))
    mag_2 = torch.sqrt(torch.sum(vector2**2))

    return dot_p / (mag_1 * mag_2)

vec1 = torch.tensor([1,2,3])
vec2 = torch.tensor([1,2,3])
vec3 = torch.tensor([-1,-2,-3])
vec4 = torch.tensor([4,5,6])


print(f"{dot_product(vec1,vec2) = }")
print(f"{dot_product(vec1,vec3) = }")
print(f"{dot_product(vec1,vec4) = }")

print(f"{cosine_similarity(vec1,vec2) = }")
print(f"{cosine_similarity(vec1,vec3) = }")
print(f"{cosine_similarity(vec1,vec4) = }")


dot_product(vec1,vec2) = tensor(14)
dot_product(vec1,vec3) = tensor(-14)
dot_product(vec1,vec4) = tensor(32)
cosine_similarity(vec1,vec2) = tensor(1.0000)
cosine_similarity(vec1,vec3) = tensor(-1.0000)
cosine_similarity(vec1,vec4) = tensor(0.9746)


In [15]:
# creating a function to get top k result from embeddings
def return_relevant_top_k(query:str,
                          embeddings:torch.tensor,
                          embedding_model:SentenceTransformer = embedding_model,
                          k:int = 5,
                          device:torch.device = device
):
    """Return a top k result from embeddings based on dot product similarity with the query
    input:
        query: str Query to be searched
        embeddings: torch.tensor Embeddings in which to search
        embedding_model: SentenceTransformer = embedding_model Embedding model
        k: int = 5
        device: torch.device = device
    output:
        top_relevant_results: list[dict]
    """
    # create embedding of query
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    # dot product
    dot_scores = util.dot_score(a=query_embedding,b=embeddings)[0]
    # getting top k index and scores
    top_k_dot_products = torch.topk(dot_scores,k=k)

    return top_k_dot_products

In [16]:
def get_relevent_top_k(top_k_dot_products:torch.tensor,pages_and_chunks_filtered:list[dict]):
    ans = []
    for score,index in zip(top_k_dot_products.values,top_k_dot_products.indices):
        relevent_chunk = pages_and_chunks_filtered[index]
        """{'page_number': 319,
        'text': "---"
        'sentence_chunk':
        'chunk_char_count': 627,
        'chunk_word_count': 96,
        'chunk_token_count': 156.75,
        'embedding':
        """
        ans.append({
            "score":score,
            "text":relevent_chunk["text"],
            "page_number":relevent_chunk["page_number"],
            "sentence_chunk":relevent_chunk["sentence_chunk"],
        })
    return ans

In [17]:
def show_results_from_top_k(top_k_dot_products:torch.tensor,pages_and_chunks_filtered:list[dict]):
    print(f"Query: {query} \n Top 5 Results:")
    for score,index in zip(top_k_dot_products.values,top_k_dot_products.indices):
        print(f"Score: {score}")
        print("Text:")
        print_wrap(pages_and_chunks_filtered[index]["text"])
        print(f"\nPage No. {pages_and_chunks_filtered[index]['page_number']}")
        print("\n")

In [18]:
top_k_results = return_relevant_top_k("Are eggs good for health?",embeddings,embedding_model)

In [19]:
show_results_from_top_k(top_k_results,pages_and_chunks_filtered)

Query: Diseases Involving Proteins 
 Top 5 Results:
Score: 0.43464937806129456
Text:
Food PDCAAS* Milk protein 1.00 Egg white 1.00 Whey 1.00 Soy protein 1.00 Beef
0.92 Soybeans 0.91 Chickpeas 0.78 Fruits 0.76 Vegetables 0.73 Whole wheat 0.42
*1 is the highest rank, 0 is the lowest Protein Needs: Special Considerations
Some groups may need to examine how to meet their protein needs more closely
than others. We will take a closer look at the special protein considerations
for vegetarians, the elderly, and athletes. Vegetarians and Vegans People who
follow variations of the vegetarian diet and consume eggs and/or dairy products
can meet their protein requirements by consuming adequate amounts of these
foods. Vegetarians and vegans can also attain their recommended protein intakes
if they give a little more attention to high-quality plant-based protein
sources. However, when following a vegetarian diet, the amino acid lysine can be
challenging to acquire. Grains, nuts, and seeds are lysine

In [20]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 15 GB


In [21]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 15 | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.
use_quantization_config set to: False
model_id set to: google/gemma-2b-it


In [22]:
# pip 1
!pip install bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.8 MB/s eta 0:00:00


In [23]:
!pip install huggingface_hub
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.utils import is_flash_attn_2_available
import torch


In [24]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [25]:
torch.cuda.get_device_capability(0)[0]

7

In [83]:

# Authenticate with your Hugging Face token
# Replace "YOUR_HF_TOKEN" with your actual token
# login(token="YOUR_HF_TOKEN")

# create quantization config
# requires !pip install bitsandbytes accelerate
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)


# 2 falsh attention is available use it for faster attention mechanism
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8.0):
    attn_implementation = "flash_attention_2"
else:
    attn_implementation = "sdpa"
print(f"Attention Implementation: {attn_implementation}")

# model id
print(model_id)

# instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it") # This line was causing the error

# instantiate model
llm_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    torch_dtype=torch.float16,
    quantization_config=quantization_config if use_quantization_config else None,
    low_cpu_mem_usage=True,
    attn_implementation=attn_implementation
)

if not use_quantization_config:
    llm_model.to(device)

Attention Implementation: sdpa
google/gemma-2b-it


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [84]:
llm_model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

In [28]:
def get_model_num_parameters(model: torch.nn.Module):
    return sum(p.numel() for p in model.parameters())

print(f"Number of parameters: {get_model_num_parameters(llm_model)}")

Number of parameters: 2506172416


In [29]:
def get_model_mem_size(model: torch.nn.Module):
  mem_params = sum(p.nelement()*p.element_size() for p in model.parameters())
  mem_buffers = sum(p.nelement()*p.element_size() for p in model.buffers())
  mem_size = mem_params + mem_buffers
  return mem_size / 2**20 # bytes to megabytes

print(f"Memory size: {get_model_mem_size(llm_model)} MB")

Memory size: 4780.1533203125 MB


## GENERATE TEXT USING LLM

In [30]:
input_text = "What are micronutrients, and what are they do in human body?"

print(f"Input Text: {input_text}")

# create prompt template
dialouge_template = [
    {
        "role":"user",
        "content":input_text
    }
]

# apply template
prompt = tokenizer.apply_chat_template(conversation=dialouge_template,
                                       tokenize=False,
                                       add_generation_prompt=True)

print(f"Prompt: {prompt}")

Input Text: What are micronutrients, and what are they do in human body?
Prompt: <bos><start_of_turn>user
What are micronutrients, and what are they do in human body?<end_of_turn>
<start_of_turn>model



In [31]:
%%time

# tokenize input text/prompt and send to GPU

input_ids = tokenizer(prompt,return_tensors="pt").to(device)

# generate output
outputs = llm_model.generate(**input_ids,max_new_tokens=256)
print(f"Output: {outputs[0]}")

Output: tensor([     2,      2,    106,   1645,    108,   1841,    708,  92800, 184592,
        235269,    578,   1212,    708,    984,    749,    575,   3515,   2971,
        235336,    107,    108,    106,   2516,    108,  21404, 235269,   1517,
        235303, 235256,    476,  13367,    576,  92800, 184592,    578,   1024,
          7257,    575,    573,   3515,   2971, 235292,    109,    688,   5057,
          1695, 184592,    688,    708,  37132,    674,    573,   2971,   2952,
        156734,    611,   1277,   1997,    578,   2004,    614,   7034,    774,
           573,  11652, 235265,   2365,    708,   8727,    604,   4282,  55786,
          7257, 235269,   3359, 235292,    109,    688, 235274, 235265,  10367,
          4584,  66058,    108, 235290,  37773,    599, 235274, 235284,    603,
         20305,    604,    573,   4584,    576,   3118,   5330,   5999, 235269,
           948,   6383,  16175,   6900,    573,   2971, 235265,    109,    688,
        235284, 235265,  41256, 

In [32]:
# output token to text
output_text = tokenizer.decode(outputs[0],skip_special_tokens=True)
print(f"Output Text: {output_text}")

Output Text: user
What are micronutrients, and what are they do in human body?
model
Sure, here's a summary of micronutrients and their functions in the human body:

**Micronutrients** are nutrients that the body cannot synthesize on its own and must be obtained from the diet. They are essential for various bodily functions, including:

**1. Energy production:**
- Vitamin B12 is crucial for the production of red blood cells, which carry oxygen throughout the body.

**2. Bone health:**
- Calcium and vitamin D are essential for strong bones and teeth.

**3. Immune function:**
- Vitamin C, vitamin A, and zinc support the immune system and protect against infections.

**4. Metabolism:**
- Vitamin B6 helps convert food into energy.
- Vitamin K supports blood clotting and bone health.

**5. Cell growth and development:**
- Micronutrients like iron, iodine, and selenium are essential for cell growth, repair, and development.

**6. Brain function:**
- Vitamin B12 is essential for nerve and bra

In [33]:
# show md output from python

from IPython.display import Markdown, display

display(Markdown(output_text))

user
What are micronutrients, and what are they do in human body?
model
Sure, here's a summary of micronutrients and their functions in the human body:

**Micronutrients** are nutrients that the body cannot synthesize on its own and must be obtained from the diet. They are essential for various bodily functions, including:

**1. Energy production:**
- Vitamin B12 is crucial for the production of red blood cells, which carry oxygen throughout the body.

**2. Bone health:**
- Calcium and vitamin D are essential for strong bones and teeth.

**3. Immune function:**
- Vitamin C, vitamin A, and zinc support the immune system and protect against infections.

**4. Metabolism:**
- Vitamin B6 helps convert food into energy.
- Vitamin K supports blood clotting and bone health.

**5. Cell growth and development:**
- Micronutrients like iron, iodine, and selenium are essential for cell growth, repair, and development.

**6. Brain function:**
- Vitamin B12 is essential for nerve and brain function.

**7. Metabolism:**
- Vitamin A is important for vision, skin health, and cell growth.
- Vitamin C helps absorb iron from food.

**8. Pregnancy and breastfeeding:**
- Micronutrients are crucial for

In [34]:
# test Retrival pipeline

queries = [
    "What are the macronutrients, and what roles do they play in the human body?",
    "How do vitamins and minerals differ in their roles and importance for health?",
    "Describe the process of digestion and absorption of nutrients in the human body.",
    "What role does fibre play in digestion? Name five fibre containing foods.",
    "Explain the concept of energy balance and its importance in weight management.",
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does saliva help with digestion?",
    "What is the RDI for protein per day?",
    "water soluble vitamins"
]

In [35]:
import random

query  = random.choice(queries)
print(f"Query: {query}")

top_k_results = return_relevant_top_k(query,embeddings,embedding_model)

top_k_results

Query: How does saliva help with digestion?


torch.return_types.topk(
values=tensor([0.6383, 0.6099, 0.5877, 0.5853, 0.5845], device='cuda:0'),
indices=tensor([364, 139, 365, 134, 366], device='cuda:0'))

In [36]:
get_relevent_top_k(top_k_results,pages_and_chunks_filtered)

[{'score': tensor(0.6383, device='cuda:0'),
  'text': 'Digestion and Absorption of Carbohydrates UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM From the Mouth to the Stomach The mechanical and chemical digestion of carbohydrates begins in the mouth. Chewing, also known as mastication, crumbles the carbohydrate foods into smaller and smaller pieces. The salivary glands in the oral cavity secrete saliva that coats the food particles. Saliva contains the enzyme, salivary amylase. This enzyme breaks the bonds between the monomeric sugar units of disaccharides, oligosaccharides, and starches. The salivary amylase breaks down amylose and amylopectin into smaller chains of glucose, called dextrins and maltose. The increased concentration of maltose in the mouth that results from the mechanical and chemical breakdown of starches in whole grains is what enhances their sweetness. Only about five percent of starches are broken down in the mouth

## Augmenting out prompts

In [80]:
def prompt_formatter(query: str,
                     context_items: list[dict]) -> str:
    # context = "- " + "\n- ".join([f"({item['score']:.2f}) {item['text']}" for item in context_items])

    # base_prompt = f"""
    # Based on the following context with there match score with the query in the starting parenthesis, Give me the answer for the query.
    # Context : {context}

    # Query : {query}

    # Answer :
    # """
    context = ", \n".join([item['text'] for item in context_items])

    base_prompt = """Please read the following context items, and then answer the question below based on the provided context, If Context is not sufficient then answer question on your own,

Context:
{context}

Question:
{query}

Answer:"""
    base_prompt = base_prompt.format(context=context,query=query)

    # create prompt template
    dialouge_template = [
        {
            "role":"user",
            "content":base_prompt
        }
    ]

    # apply template
    prompt = tokenizer.apply_chat_template(conversation=dialouge_template,
                                          tokenize=False,
                                          add_generation_prompt=True)

    # print(f"Prompt: {prompt}")

    return prompt

query = random.choice(queries)
print(f"Query: {query}")

top_k_results = return_relevant_top_k(query,embeddings,embedding_model)

context_items = get_relevent_top_k(top_k_results,pages_and_chunks_filtered)

prompt = prompt_formatter(query,context_items)

print(prompt)

Query: How often should infants be breastfed?
<bos><start_of_turn>user
Please read the following context items, and then answer the question below based on the provided context, If Context is not sufficient then answer question on your own,

Context:
milk is the best source to fulfill nutritional requirements. An exclusively breastfed infant does not even need extra water, including in hot climates. A newborn infant (birth to 28 days) requires feedings eight to twelve times a day or more. Between 1 and 3 months of age, the breastfed infant becomes more efficient, and the number of feedings per day often become fewer even though the amount of milk consumed stays the same. After about six months, infants can gradually begin to consume solid foods to help meet nutrient needs. Foods that are added in addition to breastmilk are called complementary foods. Complementary foods should be nutrient dense to provide optimal nutrition. Complementary foods include baby meats, vegetables, fruits, in

In [75]:
%%time

input_ids = tokenizer(prompt,return_tensors="pt").to(device)

# generate output
outputs = llm_model.generate(**input_ids,temperature=0.5,max_new_tokens=512,do_sample=True)

# output token to text
output_text = tokenizer.decode(outputs[0])
print(f"Query: {query}")
print(f"Output Text: {output_text.replace(prompt,'')}")

Query: What are symptoms of pellagra?
Output Text: <bos>Symptoms of pellagra include diarrhea, dermatitis, dementia, and sometimes death.<eos>
CPU times: user 867 ms, sys: 1.89 ms, total: 869 ms
Wall time: 889 ms


In [76]:
# Creating every thing in one function like query -> output chat bot type for pdf

In [77]:
def ask(
    query:str,
    temperature:float=0.7,
    max_new_tokens:int=512,
    k:int = 5,
    device:torch.device = device if device else torch.device("cpu")
) -> str:
    # get top k results ie retrival
    top_k_results = return_relevant_top_k(query,embeddings,embedding_model,k=k)
    context_items = get_relevent_top_k(top_k_results,pages_and_chunks_filtered)

    # make prompt
    prompt = prompt_formatter(query,context_items)
    print(prompt)

    # tokenize
    input_ids = tokenizer(prompt,return_tensors="pt").to(device)

    # generate output
    outputs = llm_model.generate(**input_ids,temperature=temperature,max_new_tokens=max_new_tokens,do_sample=True)

    # output token to text
    output_text = tokenizer.decode(outputs[0])

    return output_text.replace(prompt,'')


In [85]:
ask("Types of proteins")

<bos><start_of_turn>user
Please read the following context items, and then answer the question below based on the provided context, If Context is not sufficient then answer question on your own,

Context:
Proteins are similar to carbohydrates and lipids in that they are polymers of simple repeating units; however, proteins are much more structurally complex. In contrast to carbohydrates, which have identical repeating units, proteins are made up of amino acids that are different from one another. Furthermore, a protein is organized into four different structural levels. Primary: The first level is the one-dimensional sequence of amino acids that are held together by peptide bonds. Carbohydrates and lipids also are one-dimensional sequences of their respective monomers, which may be branched, coiled, fibrous, or globular, but their conformation is much more random and is not organized by their sequence of monomers. Secondary: The second level of protein structure is dependent on the che

"<bos>The provided context describes the following types of proteins:\n\n* **Fibrous proteins:** These are larger, more difficult to digest proteins that are often indigestible. \n* **Globular proteins:** Smaller, more easily digested proteins. \n \nThe context also mentions that proteins can be composed of **amino acids**, which are the building blocks of proteins.\n\n\nLet me know if you'd like more details on any of these concepts. \n<end_of_turn>"

In [79]:
def direct_llm(prompt):
    # tokenize
    input_ids = tokenizer(prompt,return_tensors="pt").to(device)

    # generate output
    outputs = llm_model.generate(**input_ids,temperature=0.7,max_new_tokens=256)

    # output token to text
    output_text = tokenizer.decode(outputs[0])

    return output_text.replace(prompt,'')

In [58]:
direct_llm("What are protiens?Are they good for human?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


"<bos>\n\nSure, here's a summary of the question and its answers:\n\n**Question:** What are protiens? Are they good for humans?\n\n**Answer:** Protiens are a diverse group of proteins found in all living organisms. They are essential for various biological processes, including metabolism, cell structure, and immune function.\n\n**Good for humans:**\n\n* **Essential for metabolism:** Protiens provide the building blocks for enzymes, which are proteins that catalyze chemical reactions in the body. Without enzymes, metabolism would not occur.\n* **Support immune function:** Antibodies, which are produced by the immune system, are primarily proteins. Protiens are also found in phagocytes, which are cells that engulf and destroy foreign particles.\n* **Maintain healthy bones and teeth:** Collagen, a protein found in bones and teeth, is largely composed of protiens.\n* **Support muscle growth and repair:** Myosin and actin, two proteins involved in muscle contraction, are both protiens.\n* *